# Review of Repo Operations

## Primary

[OFR](https://www.financialresearch.gov/)

* Hedge Funds
  - size - gross notional exposure by asset class (USD)
  - size - gross notional exposure by strategy class (USD)
  - leverage - borrowing by type (USD)
  - leverage - long and short repo exposures (USD)
* UM Money Market Fund Monitor
  - Fed Res Repo facility total utilization and MMF's participation
* Short-term Funding Monitor
  - Repo transaction volume by venue
  - Tri-party repo transation volumen collateral type
  - Money market fund investment holding by asset type
  - US Repo Mkts - Tri-party repo transaction volume collateral type
  - US Repo Mkts - DVP Service Average Rate: Total (final)
  - US Repo Mkts - GCF Repo Service Average Rate: Total (final)
  - US Repo Mkts - Triparty Average Rate: Total (final)
  - Repo tranaction volumes by venue
  - Reference rates: Effective Federal Funds Rate
  - Primary Dealer Statistics: Primary Dealer Aggregate Fails to Deliver (total)

[Federal Reserve NY](https://www.newyorkfed.org/markets/data-hub)

* Fed Res Repo Ops - Submitted, Accepted, Rate
* Fed Res Repo Ops - Submitted, Accepted, Rate (by security type Treaury, Agency, MBS)
* Fed Res Balance Sheet (H.4.1)
  - Reserves
  - US Treasuries
  - Agency Securities
  - MBS
* Fed Res Balance Sheet
  - Loans - Primary Credit
  - Loans - Secondary Credit
* Fed Res Balance Sheet
  - Central Bank Liquidity Swaps

[DTCC](dtcc.com/charts/membership)

* Spaonsored DVP and Sponsored GC Activity
* Sponsored Repo and Sponsored Reverse Repo Activity



## Additional

[US Treasury Month Debt](https://fiscaldata.treasury.gov/datasets/monthly-statement-public-debt/summary-of-treasury-securities-outstanding)

* Bills outstanding
* ???

[SIFMA](https://www.sifma.org/resources/research/statistics/us-treasury-securities-statistics/#:~:text=Issuance%20(as%20of%20February)%20$5.2,%2C%20+6.8%25%20Y/Y)

* US Treasuries Security Issuance


FRED

* Money Market Funds; Total Financial Assets, Level (MMMFFAQ027S)
* Overnight Reverse Repurchase Agreements: Treasury Securities Sold by the Federal Reserve in the Temporary Open Market Operations (RRPONTSYD)
* Liabilities and Capital: Liabilities: Deposits with F.R. Banks, Other Than Reserve Balances: U.S. Treasury, General Account: Week Average (WTREGEN)
* Liabilities and Capital: Other Factors Draining Reserve Balances: Reserve Balances with Federal Reserve Banks: Wednesday Level (WRBWFRBL)
* SOFR
  - Secured Overnight Financing Rate (SOFR)
  - Secured Overnight Financing Rate: 99th Percentile (SOFR99, SOFR75, SOFR25, SOFR1
  - Secured Overnight Financing Volume (SOFRVOL)
* FedFunds
  - Federal Funds Effective Rate (FEDFUNDS)
  - Effective Federal Funds Volume (EFFRVOL)
* Overnight
  - Overnight Bank Funding Volume (OBFRVOL)
* Calculated
  - SOFR Volume / Liabilities and Capital: Other Factors Draining Reserve Balances

Yahoo

* SP500
* Price gold
* Price silver
* calc: sp500 / gold
* calc: gold / silver


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#project
from macroecon_wrappy import (
    Auth,
    Metric,
    Measure,
    Epoch,
    plots
)
from macroecon_wrappy.adapters import (
    FredApi, 
    TreasuryFiscal
)
from macroecon_wrappy.extractors import (
    NberExtract
)


#external
from fredapi import Fred
from treasury.client import FederalTreasuryClient


#system
from pathlib import Path

In [3]:
secrets_path = Path('/workspaces/API-macroecon-wrappy/SECRETS.yaml')
cache_path = Path('/workspaces/API-macroecon-wrappy/tests/tmp/')
auth = Auth(secrets_path, cache_path)
auth.load_secrets()

True

In [4]:
#nber
NberExtract.set_config(auth)
name = 'recessions'
recessions = NberExtract.get_data(name)

first two columns will be renamed: ['start', 'end', 'name']


In [5]:
#fred
#source: https://fred.stlouisfed.org/source?soid=23
# get this api: https://fiscaldata.treasury.gov/datasets/monthly-statement-public-debt/summary-of-treasury-securities-outstanding
# compare with sifma: https://www.sifma.org/resources/research/statistics/us-treasury-securities-statistics/#:~:text=SIFMA%20Research%20tracks%20U.S.%20Treasury,%2C%20+6.8%25%20Y/Y
FredApi.set_wrapper(auth, Fred)
mmftfa = FredApi.get_data('MMMFFAQ027S')
tb1yr = FredApi.get_data('DTB1YR')                  #rate - not what we want
treasfinancing = FredApi.get_data('MTSMFBP133FMS')  #amount of all debt - we actually want Tbills from fiscal site: 

In [6]:
TreasuryFiscal.set_wrapper(auth, FederalTreasuryClient)
debt_outstanding = TreasuryFiscal.get_data('other_data-historical_debt_outstanding')

In [14]:
measure = Measure([mmftfa, debt_outstanding], recessions)
measure.df().head(3)     #TODO:measure display should include colnames
measure

                col-0              col-1
timestamp                               
2024-10-01  7243180.0                NaN
2024-09-30        NaN  35464673929171.69
2024-07-01  6839054.0                NaN
2024-04-01  6548352.0                NaN
2024-01-01  6440700.0                NaN

Money Markets are Reverse Repos and T-bills.  MM Funds invest there for a little interest.  If there is less T-bills then more Repos are needed to fullfil the MM needs.

In [11]:
from macroecon_wrappy import plots
charts = plots.graph_ts_js(measure, cols=['col-0', 'col-1'], recession_bars=False, log_scale=None)
charts

alt.VConcatChart(...)

In [13]:
#TODO:add to plot
import altair as alt

base = alt.Chart(measure.df(interpolated=True).reset_index(), width=600, height=200).encode(x='timestamp:T')
line1 = base.mark_line(stroke='blue').encode(alt.Y('col-0'))#.transform_filter('isValid(datum.line)')
line2 = base.mark_line(stroke='orange').encode(alt.Y('col-1'))
(line1 + line2).resolve_scale(y = 'independent')

alt.LayerChart(...)